In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
# Load the metadata CSV file
metadata_path = '8k.csv'
metadata = pd.read_csv(metadata_path)

In [3]:
metadata

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
...,...,...,...,...,...,...,...,...
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn


In [4]:
# Load audio data and extract features
def extract_features(file_path):
    audio_data, _ = librosa.load(file_path, res_type='kaiser_fast')
    # Perform Fourier transform
    stft = np.abs(librosa.stft(audio_data))
    stft_scaled = np.mean(stft, axis=1)
    return stft_scaled

In [5]:
# Preprocess audio and extract features for all files
features = []
labels = []

In [6]:
audio_folder = 'audio'

In [7]:
for folder in os.listdir(audio_folder):
    if folder.startswith('fold'):
        folder_path = os.path.join(audio_folder, folder)
        for file in os.listdir(folder_path):
            if file.endswith('.wav'):
                file_path = os.path.join(folder_path, file)
                class_id = int(file.split('-')[1])
                features.append(extract_features(file_path))
                labels.append(class_id)

c:\users\soham\appdata\local\programs\python\python39\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
c:\users\soham\appdata\local\programs\python\python39\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
c:\users\soham\appdata\local\programs\python\python39\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(


In [8]:
# Convert features and labels to numpy arrays
X = np.array(features) 
y = np.array(labels)

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using KNN, Neighbours = 3 and Metric = cosine

In [10]:
# Perform classification using KNN, metric = cosine and neighbours = 3
knn1 = KNeighborsClassifier(n_neighbors=3, metric='cosine')
knn1.fit(X_train, y_train)

KNeighborsClassifier(metric='cosine', n_neighbors=3)

In [11]:
# Make predictions on the test set
y_pred = knn1.predict(X_test)

In [12]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[205   0   1   0   0   2   0   0   2   1]
 [  0  61   5   0   0   3   3   1   0   4]
 [  4   3 158   5   0   3  13   1   6   4]
 [  4   2  29 154   6   2   6   1  11   2]
 [  4   0   5   4 178   1   3   6   0   1]
 [  1   0   3   0   0 188   0   0   0   1]
 [  2   0   5   1   0   0  62   1   0   1]
 [  0   0   3   0   2   0   0 204   0   0]
 [  0   0   5   2   0   1   1   1 170   0]
 [ 11   1  29   9   1   3   3   4   3 125]]


In [13]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8614768174012593


In [14]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn1.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours = 3 and Metric = euclidean

In [15]:
# Perform classification using KNN, metric = euclidean, neighbors = 3
knn2 = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn2.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=3)

In [16]:
# Make predictions on the test set
y_pred = knn2.predict(X_test)

In [17]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[200   0   7   0   0   1   0   1   2   0]
 [  4  51   7   4   1   2   2   4   2   0]
 [  6   3 159  10   1   1   6   1   7   3]
 [  8   3  32 141   7   2  10   1  11   2]
 [  5   2  11   2 172   0   1   6   0   3]
 [  4   1   3   1   0 184   0   0   0   0]
 [  2   0  11   1   1   0  57   0   0   0]
 [  2   0   0   1   3   0   0 202   0   1]
 [  0   0   5   5   0   1   0   0 169   0]
 [ 16   3  27   8   0   3   2   2   3 125]]


In [18]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8357183743560389


In [19]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn2.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours = 3 and Metric = chebyshev

In [20]:
# Perform classification using KNN, metric = chebyshev , neighbors = 3
knn3 = KNeighborsClassifier(n_neighbors=3, metric='chebyshev')
knn3.fit(X_train, y_train)

KNeighborsClassifier(metric='chebyshev', n_neighbors=3)

In [21]:
# Make predictions on the test set
y_pred = knn3.predict(X_test)

In [22]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[189   1   8   2   0   1   1   6   2   1]
 [  2  40   7   3   7   2   2   6   5   3]
 [  7   2 147  11   5   2   8   3  10   2]
 [  5   5  56 101   8   1  17   3  17   4]
 [ 11   3  21  13 136   0   3   9   3   3]
 [  2   1   3   2   3 176   1   1   2   2]
 [  2   1  12   4   0   1  44   4   3   1]
 [  1   0   0   0   3   1   1 201   0   2]
 [  2   0  20  11   8   3   1   3 132   0]
 [ 17   5  32   9  14   7   7  18   7  73]]


In [23]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7092157985117344


In [24]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn3.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 3


# Using KNN, Neighbours = 2 and Metric = cosine

In [25]:
# Perform classification using KNN, metric = cosine and neighbours = 2
knn4 = KNeighborsClassifier(n_neighbors=2, metric='cosine')
knn4.fit(X_train, y_train)

KNeighborsClassifier(metric='cosine', n_neighbors=2)

In [26]:
# Make predictions on the test set
y_pred = knn4.predict(X_test)

In [27]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[208   0   1   0   0   0   0   0   2   0]
 [  0  68   4   0   0   2   2   0   0   1]
 [  3   4 169   8   0   2   7   0   3   1]
 [  4   2  36 156   4   3   7   0   5   0]
 [  3   0   7   6 183   0   1   2   0   0]
 [  1   1   3   0   0 187   0   0   0   1]
 [  2   0   5   1   1   0  63   0   0   0]
 [  0   0   2   1   4   0   0 202   0   0]
 [  1   0   7   2   0   1   1   1 167   0]
 [  8   0  36  11   0   4   3   7   6 114]]


In [28]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8683457355466514


In [29]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn4.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours = 2 and Metric = euclidean

In [30]:
# Perform classification using KNN, metric = euclidean, neighbors = 2
knn5 = KNeighborsClassifier(n_neighbors=2, metric='euclidean')
knn5.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=2)

In [31]:
# Make predictions on the test set
y_pred = knn5.predict(X_test)

In [32]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[202   0   5   0   0   1   0   2   1   0]
 [  4  60   5   2   0   2   2   0   2   0]
 [  4   2 172   8   0   1   4   2   4   0]
 [  7   3  43 149   6   1   5   1   2   0]
 [  5   2   9   5 177   0   1   2   0   1]
 [  4   1   3   0   0 185   0   0   0   0]
 [  2   2  11   3   1   0  53   0   0   0]
 [  1   0   0   0   5   0   0 202   0   1]
 [  0   0   6   6   0   1   1   0 166   0]
 [ 16   2  32  12   0   2   5   4   6 110]]


In [33]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8448769318832284


In [34]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn5.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours = 2 and Metric = chebyshev

In [35]:
# Perform classification using KNN, metric = chebyshev , neighbors = 2
knn6 = KNeighborsClassifier(n_neighbors=2, metric='chebyshev')
knn6.fit(X_train, y_train)

KNeighborsClassifier(metric='chebyshev', n_neighbors=2)

In [36]:
# Make predictions on the test set
y_pred = knn6.predict(X_test)

In [37]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[189   1   8   3   0   2   1   6   1   0]
 [  1  45   6   4   5   2   3   5   4   2]
 [  7   3 146  18   6   3   7   2   5   0]
 [  6   4  65 113   6   2   9   2   7   3]
 [  9   2  23  12 144   0   3   6   2   1]
 [  1   1   4   5   2 178   0   1   1   0]
 [  2   2  14   5   0   1  45   2   1   0]
 [  3   0   1   1   6   0   1 195   0   2]
 [  2   0  28   9   8   2   6   3 122   0]
 [ 13   4  40  12  15   7  11  18  13  56]]


In [38]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7057813394390383


In [39]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn6.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 3


# Using KNN, Neighbours = 1 and Metric = cosine

In [40]:
# Perform classification using KNN, metric = cosine and neighbours = 1
knn7 = KNeighborsClassifier(n_neighbors=1, metric='cosine')
knn7.fit(X_train, y_train)

KNeighborsClassifier(metric='cosine', n_neighbors=1)

In [41]:
# Make predictions on the test set
y_pred = knn7.predict(X_test)

In [42]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[208   0   1   0   0   0   0   0   2   0]
 [  0  66   3   0   0   3   1   0   0   4]
 [  1   3 155  12   0   4  11   1   5   5]
 [  2   1  15 171   4   3  10   0  10   1]
 [  0   0   3   2 191   0   2   3   0   1]
 [  0   1   3   0   0 187   0   0   0   2]
 [  1   0   4   0   0   0  66   0   0   1]
 [  0   0   0   0   4   0   0 204   0   1]
 [  0   0   3   1   0   0   0   0 176   0]
 [  1   1  16   8   0   5   3   2   3 150]]


In [43]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9009730967372639


In [44]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn7.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours = 1 and Metric = euclidean

In [45]:
# Perform classification using KNN, metric = cosine and neighbours = 1
knn8 = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn8.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=1)

In [46]:
# Make predictions on the test set
y_pred = knn8.predict(X_test)

In [47]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[200   0   2   1   0   1   0   1   4   2]
 [  3  57   5   2   1   2   2   3   2   0]
 [  3   0 156  16   0   1   7   2   5   7]
 [  4   1  23 159   6   1   8   1  12   2]
 [  2   0   5   3 185   0   2   3   0   2]
 [  3   0   2   0   0 188   0   0   0   0]
 [  1   1   5   4   1   0  56   0   2   2]
 [  0   0   0   0   5   0   0 202   0   2]
 [  0   0   3   3   0   0   1   0 172   1]
 [  7   0  17   5   0   1   3   1   6 149]]


In [48]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8723526044647968


In [49]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn8.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours = 1 and Metric = chebyshev

In [50]:
# Perform classification using KNN, metric = chebyshev , neighbors = 1
knn9 = KNeighborsClassifier(n_neighbors=1, metric='chebyshev')
knn9.fit(X_train, y_train)

KNeighborsClassifier(metric='chebyshev', n_neighbors=1)

In [51]:
# Make predictions on the test set
y_pred = knn9.predict(X_test)

In [52]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[187   1   7   3   0   1   2   6   2   2]
 [  0  44   5   3   5   2   4   5   6   3]
 [  5   1 121  21   5   3  19   4   8  10]
 [  3   2  35 118   9   2  18   4  18   8]
 [  3   1  10   8 156   1   6   8   4   5]
 [  0   0   1   5   2 182   1   1   0   1]
 [  1   1  10   2   0   0  50   3   2   3]
 [  0   0   0   0   5   1   0 198   0   5]
 [  0   0  12   8   1   1   2   2 152   2]
 [ 10   1  24   4  10   5  10  13  16  96]]


In [53]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7464224384659416


In [54]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn9.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours =  4 Metric = cosine

In [55]:
# Perform classification using KNN, metric = cosine , neighbors = 4
knn10 = KNeighborsClassifier(n_neighbors=4, metric='cosine')
knn10.fit(X_train, y_train)

KNeighborsClassifier(metric='cosine', n_neighbors=4)

In [56]:
# Make predictions on the test set
y_pred = knn10.predict(X_test)

In [57]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[201   0   1   0   0   3   1   1   2   2]
 [  0  61   6   0   0   3   2   1   0   4]
 [  4   1 156   7   1   3  15   3   4   3]
 [  3   2  33 149   5   3   9   1  10   2]
 [  3   0   8   6 177   1   3   4   0   0]
 [  2   1   3   0   0 186   0   0   0   1]
 [  2   0   8   0   0   0  61   1   0   0]
 [  1   0   4   0   3   0   0 201   0   0]
 [  1   0   5   2   0   1   2   0 168   1]
 [ 12   0  28  11   1   2   3   7   5 120]]


In [58]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8471665712650257


In [59]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn10.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours =  4 Metric = euclidean

In [60]:
# Perform classification using KNN, metric = euclidean , neighbors = 4
knn11 = KNeighborsClassifier(n_neighbors=4, metric='euclidean')
knn11.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=4)

In [61]:
# Make predictions on the test set
y_pred = knn11.predict(X_test)

In [62]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[198   0   7   0   0   2   0   2   2   0]
 [  5  49   8   3   1   2   3   4   2   0]
 [  5   2 167   6   1   2   5   1   3   5]
 [  6   1  42 133   6   2  11   2  11   3]
 [  7   1  15   4 167   0   2   4   0   2]
 [  3   1   3   0   0 185   0   0   1   0]
 [  3   2  11   1   1   0  51   1   1   1]
 [  3   0   0   1   3   0   0 201   0   1]
 [  0   0   9   5   0   1   0   0 165   0]
 [ 18   1  38   8   2   4   5   5   1 107]]


In [63]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8145392100744133


In [64]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn11.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours =  4 Metric =  chebyshev

In [65]:
# Perform classification using KNN, metric = chebyshev , neighbors = 4
knn12 = KNeighborsClassifier(n_neighbors=1, metric='chebyshev')
knn12.fit(X_train, y_train)

KNeighborsClassifier(metric='chebyshev', n_neighbors=1)

In [66]:
# Make predictions on the test set
y_pred = knn12.predict(X_test)

In [67]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[187   1   7   3   0   1   2   6   2   2]
 [  0  44   5   3   5   2   4   5   6   3]
 [  5   1 121  21   5   3  19   4   8  10]
 [  3   2  35 118   9   2  18   4  18   8]
 [  3   1  10   8 156   1   6   8   4   5]
 [  0   0   1   5   2 182   1   1   0   1]
 [  1   1  10   2   0   0  50   3   2   3]
 [  0   0   0   0   5   1   0 198   0   5]
 [  0   0  12   8   1   1   2   2 152   2]
 [ 10   1  24   4  10   5  10  13  16  96]]


In [68]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7464224384659416


In [69]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn12.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours =  5 Metric = cosine

In [70]:
# Perform classification using KNN, metric = cosine , neighbors = 5
knn13 = KNeighborsClassifier(n_neighbors=5, metric='cosine')
knn13.fit(X_train, y_train)

KNeighborsClassifier(metric='cosine')

In [71]:
# Make predictions on the test set
y_pred = knn13.predict(X_test)

In [72]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[202   0   1   0   0   2   2   1   2   1]
 [  0  58   6   0   0   4   3   2   0   4]
 [  6   1 146   9   3   3  17   5   5   2]
 [  7   2  33 141   6   3   8   1  14   2]
 [  4   0   7   3 177   1   3   7   0   0]
 [  2   1   3   1   0 186   0   0   0   0]
 [  1   0   8   0   0   0  62   1   0   0]
 [  3   0   2   0   3   0   0 201   0   0]
 [  0   0   6   2   0   2   2   0 168   0]
 [ 11   0  33  12   1   3   2   6   5 116]]


In [73]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8340011448196909


In [74]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn13.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours =  5 Metric = euclidean

In [75]:
# Perform classification using KNN, metric = euclidean , neighbors = 5
knn14 = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn14.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean')

In [76]:
# Make predictions on the test set
y_pred = knn14.predict(X_test)

In [77]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[196   0   9   0   0   0   1   2   2   1]
 [  5  45  11   2   2   3   3   5   1   0]
 [  5   1 164   5   1   3   5   2   5   6]
 [  7   2  36 130   8   4  11   2  12   5]
 [  5   2  14   6 166   1   3   4   0   1]
 [  3   1   2   1   0 184   0   0   1   1]
 [  3   0  13   1   1   0  52   1   1   0]
 [  3   0   0   0   3   0   0 202   0   1]
 [  0   0   9   5   0   1   1   0 164   0]
 [ 19   0  33   9   2   4   5   5   3 109]]


In [78]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8082427017744706


In [79]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn14.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours =  5 Metric = chebyshev

In [80]:
# Perform classification using KNN, metric = euclidean , neighbors = 5
knn15 = KNeighborsClassifier(n_neighbors=5, metric='chebyshev')
knn15.fit(X_train, y_train)

KNeighborsClassifier(metric='chebyshev')

In [81]:
# Make predictions on the test set
y_pred = knn15.predict(X_test)

In [82]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[187   0   7   3   1   0   0   8   5   0]
 [  3  36   6   4   8   1   3   7   5   4]
 [  4   0 146   9   8   3  11   4  11   1]
 [  6   2  56  98   9   2  21   6  13   4]
 [ 10   3  19  12 130   2   6  10   6   4]
 [  3   1   2   2   3 177   1   1   1   2]
 [  1   2  14   2   0   1  43   4   5   0]
 [  2   0   1   1   4   1   0 197   2   1]
 [  2   0  22  11   8   2   1   5 128   1]
 [ 13   3  35   6  13   8  11  21  15  64]]


In [83]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6903262736119061


In [84]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn15.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 4


# Using KNN, Neighbours =  6 Metric = cosine

In [85]:
# Perform classification using KNN, metric = cosine , neighbors = 6
knn16 = KNeighborsClassifier(n_neighbors=6, metric='cosine')
knn16.fit(X_train, y_train)

KNeighborsClassifier(metric='cosine', n_neighbors=6)

In [86]:
# Make predictions on the test set
y_pred = knn16.predict(X_test)

In [87]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[201   0   3   0   0   2   1   1   2   1]
 [  0  57   8   0   0   4   3   3   0   2]
 [  8   1 141   7   4   3  22   3   6   2]
 [  6   2  41 134   5   5   8   1  13   2]
 [  5   0   9   4 174   1   3   6   0   0]
 [  4   1   4   3   0 181   0   0   0   0]
 [  1   0  10   0   0   0  60   1   0   0]
 [  4   0   1   0   3   0   0 201   0   0]
 [  1   0   7   2   0   2   2   1 165   0]
 [ 11   0  36  11   1   3   3   8   5 111]]


In [88]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.815684029765312


In [89]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn16.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours =  6 Metric = euclidean

In [90]:
# Perform classification using KNN, metric = euclidean, neighbors = 6
knn17 = KNeighborsClassifier(n_neighbors=6, metric='euclidean')
knn17.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=6)

In [91]:
# Make predictions on the test set
y_pred = knn17.predict(X_test)

In [92]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[195   0  10   0   0   1   0   1   3   1]
 [  6  40  11   1   4   3   5   5   2   0]
 [  9   0 164   4   1   4   5   2   3   5]
 [  8   2  36 131   8   4   9   2  13   4]
 [  5   2  15   4 166   1   3   4   1   1]
 [  2   1   3   0   0 184   0   0   2   1]
 [  3   0  15   0   2   0  50   1   1   0]
 [  2   0   1   1   4   0   0 200   1   0]
 [  0   0  11   5   0   1   0   1 162   0]
 [ 18   2  38  10   2   5   4   7   3 100]]


In [93]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7967945048654836


In [94]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn17.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours =  6 Metric = chebyshev

In [95]:
# Perform classification using KNN, metric = chebyshev , neighbors = 6
knn18 = KNeighborsClassifier(n_neighbors=6, metric='chebyshev')
knn18.fit(X_train, y_train)

KNeighborsClassifier(metric='chebyshev', n_neighbors=6)

In [96]:
# Make predictions on the test set
y_pred = knn18.predict(X_test)

In [97]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[186   0   7   3   1   2   0   8   4   0]
 [  2  38   7   2   8   1   3   7   6   3]
 [  4   1 148   8   7   3   9   4  12   1]
 [  9   4  53  95  10   3  16   6  16   5]
 [ 10   2  24  16 122   3   4  11   3   7]
 [  4   1   4   1   2 175   1   2   1   2]
 [  1   2  15   1   0   2  44   4   3   0]
 [  2   0   1   1   4   1   1 196   3   0]
 [  2   0  27   8   8   2   2   5 126   0]
 [ 16   3  32   6  12   8  14  22  15  61]]


In [98]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.681740125930166


In [99]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn18.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 4


# Using KNN, Neighbours =  20 Metric = cosine

In [100]:
# Perform classification using KNN, metric = chebyshev , neighbors = 6
knn19= KNeighborsClassifier(n_neighbors=6, metric='cosine')
knn19.fit(X_train, y_train)

KNeighborsClassifier(metric='cosine', n_neighbors=6)

In [101]:
# Make predictions on the test set
y_pred = knn19.predict(X_test)

In [102]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[201   0   3   0   0   2   1   1   2   1]
 [  0  57   8   0   0   4   3   3   0   2]
 [  8   1 141   7   4   3  22   3   6   2]
 [  6   2  41 134   5   5   8   1  13   2]
 [  5   0   9   4 174   1   3   6   0   0]
 [  4   1   4   3   0 181   0   0   0   0]
 [  1   0  10   0   0   0  60   1   0   0]
 [  4   0   1   0   3   0   0 201   0   0]
 [  1   0   7   2   0   2   2   1 165   0]
 [ 11   0  36  11   1   3   3   8   5 111]]


In [103]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.815684029765312


In [104]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn19.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours = 20 Metric = euclidean

In [105]:
# Perform classification using KNN, metric = chebyshev , neighbors = 6
knn20= KNeighborsClassifier(n_neighbors=20, metric="euclidean")
knn20.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=20)

In [106]:
# Make predictions on the test set
y_pred = knn20.predict(X_test)

In [107]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[186   0  17   0   0   3   0   1   2   2]
 [  3  28  20   1   5   4   4  11   0   1]
 [ 11   0 159   2   2   6   4   3   9   1]
 [ 11   0  56  93   7   5   6   3  24  12]
 [ 11   0  27   4 133   3   3  17   0   4]
 [ 11   0   9   1   0 163   0   4   4   1]
 [  4   0  15   1   3   0  43   2   3   1]
 [  6   0   3   0   2   0   0 197   1   0]
 [  1   0  31   6   0   3   0   4 135   0]
 [ 20   1  58   8   6   5   1  13   5  72]]


In [108]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6920435031482541


In [109]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn20.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Using KNN, Neighbours = 20 Metric = chebyshev

In [173]:
# Perform classification using KNN, metric = chebyshev , neighbors = 6
knn21= KNeighborsClassifier(n_neighbors=20, metric="chebyshev")
knn21.fit(X_train, y_train)

KNeighborsClassifier(metric='chebyshev', n_neighbors=20)

In [174]:
# Make predictions on the test set
y_pred = knn21.predict(X_test)

In [175]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

ValueError: multilabel-indicator is not supported

In [ ]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = knn21.predict(audio_features)[0]
print("Predicted class:", predicted_class)

# SVM - RBF kernel

In [116]:
# Perform classification using Support Vector Machines (SVM) with adjusted parameters
svm1 = SVC(kernel='rbf', C=100, gamma='scale', random_state=42)
svm1.fit(X_train, y_train)

SVC(C=100, random_state=42)

In [117]:
# Make predictions on the test set
y_pred = svm1.predict(X_test)

In [118]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[185   0  12   1   4   1   0   1   4   3]
 [  0  63   3   3   0   1   1   0   1   5]
 [  5   0 145  14   0   3   4   1  21   4]
 [  7   7  28 143   4   3   0   1  18   6]
 [  1   2  11   3 179   0   2   4   0   0]
 [  3   1   7   0   1 179   0   0   1   1]
 [  1   2  10   2   1   0  51   3   1   1]
 [  3   0   7   1   7   0   0 188   0   3]
 [  2   1   9   3   0   2   0   0 163   0]
 [  7   1  24   6   7  10   0   1   4 129]]


In [119]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.815684029765312


In [120]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = svm1.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# SVM - Linear Kernel

In [121]:
# Perform classification using Support Vector Machines (SVM) with adjusted parameters
svm2 = SVC(kernel="linear", C=100, gamma='scale', random_state=42)
svm2.fit(X_train, y_train)

SVC(C=100, kernel='linear', random_state=42)

In [122]:
# Make predictions on the test set
y_pred = svm2.predict(X_test)

In [123]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[184   0   8   2   6   6   0   2   1   2]
 [  1  63   2   4   0   1   0   1   1   4]
 [  4   5 127  20   5   5   8   1  14   8]
 [  5   3  34 119  11   5   9   2  22   7]
 [  4   4  13   5 151   1   3  16   2   3]
 [  9   2  10  11   3 153   0   0   1   4]
 [  1   1  13  12   5   2  33   3   1   1]
 [  3   0   5   2  21   2   2 170   0   4]
 [  1   3   8  11   1   6   2   1 145   2]
 [ 10   8  17   6   7  10   6   3   5 117]]


In [124]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7223812249570692


In [125]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = svm2.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# SVM - Sigmoid Kernel

In [126]:
# Perform classification using Support Vector Machines (SVM) with adjusted parameters
svm3 = SVC(kernel="sigmoid", C=100, gamma='scale', random_state=42)
svm3.fit(X_train, y_train)

SVC(C=100, kernel='sigmoid', random_state=42)

In [127]:
# Make predictions on the test set
y_pred = svm3.predict(X_test)

In [128]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[ 65   0  44   2   5  21   0   4   2  68]
 [ 10  22   2   2   2   5   0   1   4  29]
 [ 30   1  61  16   6   9   0   1  25  48]
 [ 46   1  21  42   6   4   2   0  33  62]
 [ 30   0  13   1  57   1   0  22   2  76]
 [ 69   0  21   0   0  36   0   7   2  58]
 [ 25   0   9   8   3   1   6   2   1  17]
 [ 50   0   1   0  26   2   0  56   0  74]
 [ 39   0  36   7   2   7   0   1  47  41]
 [ 29   0  22   7   7   9   0   5   6 104]]


In [129]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.28391528334287347


In [130]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = svm3.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# SVM - Polynomial Kernel

In [131]:
# Perform classification using Support Vector Machines (SVM) with polynomial kernel
svm4 = SVC(kernel='poly', degree=1, random_state=42)
svm4.fit(X_train, y_train)

SVC(degree=1, kernel='poly', random_state=42)

In [132]:
# Make predictions on the test set
y_pred = svm4.predict(X_test)

In [133]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[ 86   0  83   3   6   7   0   2   0  24]
 [  3  45  15   3   4   1   0   2   0   4]
 [  7   0 164   6   5   4   0   0   0  11]
 [  2   2  92  94   5   4   1   1   7   9]
 [  8   1  25  11 125   1   0  18   0  13]
 [ 15   0  50   5   3  93   0   6   2  19]
 [  1   0  26   4   5   2  31   1   0   2]
 [ 10   0  10   0  20   9   0 150   0  10]
 [  4   0  76  10   0   4   0   3  79   4]
 [ 14   1  56   7   6  11   0   6   5  83]]


In [134]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5437893531768746


In [135]:
# Load the audio file to predict its class
audio_file = 'mixkit-horde-of-barking-dogs-60.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = svm4.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 3


# Random Forest Classifier 

In [136]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [137]:
# Make predictions on the test set
y_pred = rfc.predict(X_test)

In [138]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[182   0   7   0   0   3   0   9   1   9]
 [  3  40   8   4   7   3   0   5   0   7]
 [  7   0 142  10   8   4   2   2   4  18]
 [  3   0  31 140   8   5   4   3   7  16]
 [  3   0   9   3 166   0   0  10   2   9]
 [  4   0   6   0   0 167   0   3   4   9]
 [  3   0  13   2   7   1  33   8   0   5]
 [  1   0   2   0   1   0   0 195   0  10]
 [  0   0  15   5   2   2   0   4 147   5]
 [ 10   0  30   6   7   7   0   6   2 121]]


In [139]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7630223239839725


In [140]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = rfc.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Random Forest Classifier with extra parameters

In [141]:
rfc1 = RandomForestClassifier(n_estimators=300, max_depth=50, min_samples_split=2, min_samples_leaf=1, random_state=42)
rfc1.fit(X_train, y_train)

RandomForestClassifier(max_depth=50, n_estimators=300, random_state=42)

In [142]:
# Make predictions on the test set
y_pred = rfc1.predict(X_test)

In [143]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[200   0   2   0   0   0   0   5   1   3]
 [  1  54   6   3   2   2   0   4   0   5]
 [  4   0 162   6   6   3   2   2   3   9]
 [  1   0  15 169   5   1   6   1   7  12]
 [  1   0   4   1 185   0   0   4   0   7]
 [  0   0   2   0   0 183   0   1   4   3]
 [  2   0   6   1   3   0  53   3   0   4]
 [  0   0   0   0   4   0   0 204   0   1]
 [  0   0   7   4   2   0   0   0 166   1]
 [  7   0  14   4   3   5   0   2   2 152]]


In [144]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8746422438465942


In [145]:
# Load the audio file to predict its class
audio_file = 'mixkit-ambulance-siren-passing-by-1650.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = rfc1.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 8


# Decision Tree

In [146]:
# Perform classification using Decision Tree
dt1 = DecisionTreeClassifier(random_state=42)
dt1.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [147]:
# Make predictions on the test set
y_pred = dt1.predict(X_test)

In [148]:
# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

Classification Matrix:
[[164   2   6   9   2   4   2   3   3  16]
 [  3  36   5   9   6   4   3   2   2   7]
 [  7   2 118  26   6   4   4   3   9  18]
 [  6   3  21 115  10   9   8   5  23  17]
 [  3   6   9  10 150   3   6   9   1   5]
 [  8   1   4   5   3 157   1   5   4   5]
 [  0   1   5   5   3   4  47   2   0   5]
 [  2   2   5   1  19   6   4 161   6   3]
 [  3   3  12   7   3   4   3   1 138   6]
 [  8   4  28  18   7  11   8   4  14  87]]


In [149]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6714367487120778


In [150]:
# Load the audio file to predict its class
audio_file = 'mixkit-horde-of-barking-dogs-60.wav'
audio_features = extract_features(audio_file)

# Reshape the audio features to match the shape expected by the classifier
audio_features = audio_features.reshape(1, -1)

# Predict the class of the audio file
predicted_class = dt1.predict(audio_features)[0]
print("Predicted class:", predicted_class)

Predicted class: 3


# Decision Tree with hyperparameter search using grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Create the Decision Tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Perform grid search with cross-validation
grid_search = GridSearchCV(dt, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best parameters
print("Best Parameters:", best_params)

# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Classification Matrix:")
print(conf_matrix)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# ANN - optimizer - adam

In [160]:
# Convert features and labels to numpy arrays
X = np.array(features)
y = np.array(labels)

In [161]:
# Normalize the input features
X = (X - np.mean(X)) / np.std(X)

In [162]:
# Convert labels to categorical
y = to_categorical(y)

In [163]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [164]:
class_labels = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']

In [165]:
# Build the ANN model
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(len(class_labels), activation='softmax'))

In [166]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [167]:
# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
110/110 [==============================] - 3s 18ms/step - loss: 1.4659 - accuracy: 0.5135 - val_loss: 1.2172 - val_accuracy: 0.5776
Epoch 2/30
110/110 [==============================] - 2s 15ms/step - loss: 1.0648 - accuracy: 0.6460 - val_loss: 1.0219 - val_accuracy: 0.6812
Epoch 3/30
110/110 [==============================] - 2s 18ms/step - loss: 0.8782 - accuracy: 0.7104 - val_loss: 0.9295 - val_accuracy: 0.6978
Epoch 4/30
110/110 [==============================] - 2s 17ms/step - loss: 0.7764 - accuracy: 0.7423 - val_loss: 0.9126 - val_accuracy: 0.7081
Epoch 5/30
110/110 [==============================] - 2s 20ms/step - loss: 0.6941 - accuracy: 0.7762 - val_loss: 0.8424 - val_accuracy: 0.7344
Epoch 6/30
110/110 [==============================] - 2s 17ms/step - loss: 0.6382 - accuracy: 0.7974 - val_loss: 0.7673 - val_accuracy: 0.7367
Epoch 7/30
110/110 [==============================] - 2s 17ms/step - loss: 0.5783 - accuracy: 0.8069 - val_loss: 0.7926 - val_accuracy: 0.7396

In [168]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Test Loss: 0.5469610691070557
Test Accuracy: 0.8723526000976562


In [169]:
# Convert one-hot encoded labels to integer labels
y_test_labels = np.argmax(y_test, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate and print the classification matrix
conf_matrix = confusion_matrix(y_test_labels, y_pred_labels)
print("Classification Matrix:")
print(conf_matrix)

AxisError: axis 1 is out of bounds for array of dimension 1